In [95]:
import pandas as pd,numpy as np
from scipy import stats
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, confusion_matrix

In [125]:
df = pd.read_csv('../0. Dataset/Training set after splitting.csv')

In [114]:
df.head()

,CustomerID,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus,Churn
0,3317770,39.79,234.00,55.00,0.25,0.00,1.70,31.00,8.80,6.70,...,0,No,0,Unknown,No,2-High,Suburban,Other,Unknown,No
1,3224894,60.19,551.00,60.00,0.74,0.00,0.30,-88.00,-10.20,2.70,...,7,No,0,150,No,2-High,Suburban,Crafts,Yes,No
2,3231266,85.41,642.00,28.00,0.25,134.00,0.00,129.00,22.80,29.70,...,4,No,0,Unknown,No,2-High,Suburban,Crafts,No,No
3,3184922,30.18,281.00,30.00,0.00,0.00,0.00,132.00,-0.20,4.00,...,3,No,0,150,No,3-Good,Suburban,Other,Unknown,Yes
4,3350054,179.29,1459.00,95.00,0.50,118.00,48.60,-1360.00,-77.00,21.70,...,4,No,0,Unknown,No,3-Good,Rural,Other,Unknown,Yes


In [115]:
m = df.copy()
m.drop(columns = ['HandsetPrice'],axis=1,inplace=True)
m.drop(columns=['CustomerID'],axis=1,inplace=True)
m['MaritalStatus'] = m['MaritalStatus'].replace('Unknown', np.nan)

Remove outliers, standarize monthly revenue, monthly 

In [98]:
df.duplicated().sum()

0

In [99]:
df.dtypes

CustomerID                     int64
MonthlyRevenue               float64
MonthlyMinutes               float64
TotalRecurringCharge         float64
DirectorAssistedCalls        float64
OverageMinutes               float64
RoamingCalls                 float64
PercChangeMinutes            float64
PercChangeRevenues           float64
DroppedCalls                 float64
BlockedCalls                 float64
UnansweredCalls              float64
CustomerCareCalls            float64
ThreewayCalls                float64
ReceivedCalls                float64
OutboundCalls                float64
InboundCalls                 float64
PeakCallsInOut               float64
OffPeakCallsInOut            float64
DroppedBlockedCalls          float64
CallForwardingCalls          float64
CallWaitingCalls             float64
MonthsInService                int64
UniqueSubs                     int64
ActiveSubs                     int64
ServiceArea                   object
Handsets                     float64
H

In [108]:
df.isna().sum()

MonthlyRevenue                 120
MonthlyMinutes                 120
TotalRecurringCharge           120
DirectorAssistedCalls          120
OverageMinutes                 120
RoamingCalls                   120
PercChangeMinutes              290
PercChangeRevenues             290
DroppedCalls                     0
BlockedCalls                     0
UnansweredCalls                  0
CustomerCareCalls                0
ThreewayCalls                    0
ReceivedCalls                    0
OutboundCalls                    0
InboundCalls                     0
PeakCallsInOut                   0
OffPeakCallsInOut                0
DroppedBlockedCalls              0
CallForwardingCalls              0
CallWaitingCalls                 0
MonthsInService                  0
UniqueSubs                       0
ActiveSubs                       0
ServiceArea                      0
Handsets                         1
HandsetModels                    1
CurrentEquipmentDays             1
AgeHH1              

In [101]:
df.drop(columns=['CustomerID'],axis=1,inplace=True)
df['HandsetPrice'] = df['HandsetPrice'].replace('Unknown', np.nan)
df['MaritalStatus'] = df['MaritalStatus'].replace('Unknown', np.nan)

In [102]:
imputer = KNNImputer(weights = 'distance')
imputer

KNNImputer(weights='distance')

In [106]:
df = df.dropna(subset=['ServiceArea'])

In [129]:
(df['ServiceArea'].value_counts()/df.shape[0])*100

NYCBRO917   3.26
HOUHOU281   3.02
DALDAL214   2.93
NYCMAN917   2.34
DALFTW817   1.56
            ... 
NCRSAN919   0.00
OHINOR419   0.00
NORZIM763   0.00
SEWPAS509   0.00
SEWMLF541   0.00
Name: ServiceArea, Length: 729, dtype: float64

In [131]:
len(df['ServiceArea'][0])

9

In [109]:
imputer.fit_transform(df)

ValueError: could not convert string to float: 'LAXSAN714'

In [38]:
df['HandsetPrice'] = df['HandsetPrice'].astype(int)
#df.reset_index(drop=True,inplace=True)
yes_no_features = ['Churn','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','BuysViaMailOrder',
     'RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCreditCard','NewCellphoneUser','NotNewCellphoneUser',
     'OwnsMotorcycle','MadeCallToRetentionTeam','MaritalStatus']
for feature in yes_no_features:
    df[feature] = [1 if value == 'Yes' else 0 for value in df[feature]]
df['Homeownership'] = [1 if value == 'Known' else 0 for value in df['Homeownership']]

In [133]:
state_map = {
        'NYC': 'New York', 'LAX': 'California', 'CHI': 'Illinois', 'HAR': 'Connecticut',
        'DAL': 'Texas', 'SAN': 'Texas', 'MIA': 'Florida', 'MIL': 'Wisconsin', 'NCR': 'North Carolina',
        'PHI': 'Pennsylvania', 'OKC': 'Oklahoma', 'MIN': 'Minnesota', 'OHI': 'Ohio', 'IND': 'Indiana',
        'SFR': 'California', 'LOU': 'Kentucky', 'APC': 'Maryland', 'BOS': 'Massachusetts', 'NOL': 'Louisiana',
        'SEA': 'Washington', 'DET': 'Michigan', 'PHX': 'Arizona', 'NEV': 'Nevada', 'OMA': 'Nebraska',
        'NNY': 'New York', 'STL': 'Missouri', 'FLN': 'Florida', 'HOU': 'Texas', 'PIT': 'Pennsylvania',
        'KCY': 'Kansas', 'ATL': 'Georgia', 'AIR': 'South Carolina', 'NMX': 'New Mexico', 'SHE': 'Virginia',
        'LAU': 'Mississippi', 'SLC': 'Utah', 'HWI': 'Hawaii', 'NSH': 'Tennessee', 'BIR': 'Alabama',
        'AWI': 'Wisconsin', 'DEN': 'Colorado', 'NOR': 'Minnesota', 'GCW': 'Mississippi', 'NVU': 'Nevada',
        'LAW': 'Texas', 'VAH': 'Virginia', 'SDA': 'South Dakota', 'SEW': 'Oregon', 'HOP': 'Montana',
        'SFU': 'California'
    }

def get_state(code):
    if not isinstance(code, str) or len(code) < 3:
        return 'Unknown'
    metro_code = code[:3]  # Extract first 3 letters (e.g., 'NYC')
    # Disambiguate 'SAN' based on area code
    if metro_code == 'SAN':
        area_code = code[-3:] if len(code) >= 9 else ''
        if area_code in ['512', '210', '830', '254', '361', '956']:  # Texas area codes
            return 'Texas'
        return 'California'  # Default for 'SAN' in other contexts (e.g., Santa Ana)
    return state_map.get(metro_code, 'Unknown')

In [121]:
keep_values = {'CreditRating': ['2-High','1-Highest','3-Good'], 'Occupation': ['Professional','Crafts'], 
               'PrizmCode': ['Suburban','Town'],'ServiceArea': ['Texas', 'New York', 'California', 'Florida',
               'Maryland', 'Illinois','Georgia', 'Ohio', 'North Carolina', 'Nevada', 'Pennsylvania','Michigan', 
               'Massachusetts' ,'Washington', 'Kansas'] }

for feature, values in keep_values.items():
    df[feature] = [value if value in values else 'Other' for value in df[feature]]

In [134]:
df['ServiceArea'] = df['ServiceArea'].apply(lambda code: get_state(code))

In [140]:
df['ServiceArea'].value_counts().index

Index(['Texas', 'New York', 'California', 'Florida', 'Maryland', 'Illinois',
       'Georgia', 'Ohio', 'North Carolina', 'Nevada', 'Pennsylvania',
       'Michigan', 'Massachusetts', 'Washington', 'Kansas', 'Connecticut',
       'Missouri', 'Minnesota', 'Arizona', 'New Mexico', 'Colorado',
       'South Carolina', 'Wisconsin', 'Oklahoma', 'Nebraska', 'Indiana',
       'Kentucky', 'Tennessee', 'Louisiana', 'Unknown', 'Hawaii', 'Virginia',
       'Utah', 'Alabama', 'Mississippi', 'Oregon', 'South Dakota', 'Montana'],
      dtype='object')

In [145]:
5109 + 5094 +4962+3026+1919+1608+1456+1437+1354+1280+1259+1238+1224 + 870 +803

32639

In [146]:
32639 / df.shape[0]

0.7992506795308176

In [141]:
df['ServiceArea'].value_counts()

Texas             5109
New York          5094
California        4962
Florida           3026
Maryland          1919
Illinois          1608
Georgia           1456
Ohio              1437
North Carolina    1354
Nevada            1280
Pennsylvania      1259
Michigan          1238
Massachusetts     1224
Washington         871
Kansas             803
Connecticut        757
Missouri           745
Minnesota          659
Arizona            618
New Mexico         597
Colorado           567
South Carolina     504
Wisconsin          503
Oklahoma           477
Nebraska           399
Indiana            386
Kentucky           365
Tennessee          327
Louisiana          301
Unknown            267
Hawaii             212
Virginia           148
Utah               127
Alabama             92
Mississippi         70
Oregon              50
South Dakota        25
Montana              1
Name: ServiceArea, dtype: int64

In [40]:
df = pd.get_dummies(df,columns=['CreditRating','PrizmCode','Occupation','ServiceArea'])

In [110]:
def selectBestFeatures(df, target):
    selected_features = []
    for column in df.columns:
        if column != target:
            corr,pval = stats.pearsonr(df[column],df[target])
            if pval < 0.05:
                selected_features.append(column)
            #print("'{column}' has correlation of '{corr}' and pvalue of '{pval}'".format(column=column,corr=corr,pval=pval))
    return selected_features

In [45]:
len(selected_features)

53

In [12]:
df

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus,Churn,MaritalStatus
0,39.79,234.0,55.0,0.25,0.0,1.7,31.0,8.8,6.7,0.7,...,0,0,NaN,0,2-High,Suburban,Other,Unknown,0,NaN
1,60.19,551.0,60.0,0.74,0.0,0.3,-88.0,-10.2,2.7,32.7,...,0,0,150,0,2-High,Suburban,Crafts,Yes,0,Yes
2,85.41,642.0,28.0,0.25,134.0,0.0,129.0,22.8,29.7,0.0,...,0,0,NaN,0,2-High,Suburban,Crafts,No,0,No
3,30.18,281.0,30.0,0.00,0.0,0.0,132.0,-0.2,4.0,0.0,...,0,0,150,0,3-Good,Suburban,Other,Unknown,1,NaN
4,179.29,1459.0,95.0,0.50,118.0,48.6,-1360.0,-77.0,21.7,11.0,...,0,0,NaN,0,3-Good,Rural,Other,Unknown,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40832,19.36,18.0,17.0,0.00,0.0,4.5,-18.0,-9.4,0.0,0.0,...,0,0,NaN,0,1-Highest,Town,Professional,Yes,1,Yes
40833,50.30,140.0,50.0,0.00,0.0,0.0,-77.0,-0.3,1.3,3.0,...,0,0,NaN,0,1-Highest,Suburban,Professional,Yes,0,Yes
40834,15.73,713.0,10.0,1.73,0.0,0.0,-143.0,0.2,4.0,55.0,...,0,0,NaN,0,3-Good,Suburban,Other,Unknown,1,NaN
40835,80.69,504.0,50.0,0.00,77.0,0.0,48.0,-30.7,1.7,0.7,...,0,0,NaN,0,4-Medium,Suburban,Other,No,0,No


In [126]:
(df['PrizmCode'].value_counts()/df.shape[0])*100

Other      48.34
Suburban   32.18
Town       14.79
Rural       4.69
Name: PrizmCode, dtype: float64

In [123]:
df['Occupation'].value_counts()

Other           32634
Professional     7001
Crafts           1202
Name: Occupation, dtype: int64

In [122]:
(df['CreditRating'].value_counts()/df.shape[0])*100

2-High      37.19
Other       29.66
1-Highest   16.71
3-Good      16.44
Name: CreditRating, dtype: float64

In [21]:
(df['HandsetPrice'].value_counts()/df.shape[0])*100

30     14.384014
150     7.995200
130     4.079634
80      3.851899
10      3.739256
60      3.501726
200     2.480594
100     2.414477
40      0.497098
400     0.090604
250     0.044078
300     0.024488
180     0.022039
500     0.017141
240     0.012244
Name: HandsetPrice, dtype: float64

In [116]:
37.1 + 16.7+ 16.4

70.19999999999999

In [32]:
def remove_outliers_using_IQR(df,column):
    first_quartile, third_quartile = np.percentile(df[column],25), np.percentile(df[column],75)
    IQR = third_quartile - first_quartile
    floor = first_quartile - 1.5 * IQR
    cieling = third_quartile + 1.5 * IQR
    outlier_mask = (df[column] < floor) | (df[column] > cieling)
    print()
    df = df[~outlier_mask]
    return df

In [47]:
column = 'BlockedCalls'
first_quartile, third_quartile = np.percentile(df[column],25), np.percentile(df[column],75)
IQR = third_quartile - first_quartile
floor = first_quartile - 1.5 * IQR
cieling = third_quartile + 1.5 * IQR
outlier_mask = (df[column] < floor) | (df[column] > cieling)
print(df[~outlier_mask][column].unique())

[0.7 0.  2.  5.7 6.7 7.3 0.3 3.  4.7 3.7 5.  9.  1.  1.7 7.  1.3 5.3 4.
 3.3 4.3 2.7 2.3 6.  8.7 8.  6.3 7.7 8.3]


In [50]:
features = df[selected_features]
labels = df['Churn']

x_train, x_validation, y_train, y_validation = train_test_split(features, labels, random_state = 42, stratify = labels, test_size = 0.2)

In [54]:
classifier = DecisionTreeClassifier()

In [55]:
classifier.fit(x_train,y_train)

DecisionTreeClassifier()

In [56]:
y_pred = classifier.predict(x_validation)

In [57]:
precision_score(y_validation,y_pred)

0.32437275985663083

In [58]:
recall_score(y_validation,y_pred)

0.32437275985663083

In [63]:
y = classifier.predict(x_train)
precision_score(y_train,y)

1.0

In [64]:
recall_score(y_train,y)

1.0

In [66]:
params = classifier.get_params()

In [68]:
params

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [77]:
#Models
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier, RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn import tree
from sklearn.linear_model import SGDClassifier,RidgeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier


models = {
    "AdaBoost" : AdaBoostClassifier(DecisionTreeClassifier(),algorithm="SAMME.R",n_estimators=100),
    "GradientBoosting"      : GradientBoostingClassifier(),
    "BaggingClassifier"  : BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)),
    "RandomForest"      : RandomForestClassifier(n_estimators=3),
    "eXtremeGradientBoosting"     : XGBClassifier(objective= 'binary:logistic'),
    #"KNN"                : KNeighborsClassifier(n_neighbors = 30, weights = 'distance'),
    "DecisionTree"      : DecisionTreeClassifier(),
    "ExtraTreeClassifier"       : ExtraTreesClassifier(n_estimators=3)
}


for name, model in models.items():
    

    print(f'Using model: {name}')
    model.fit(x_train, y_train)

    print(f'Training Score: {model.score(x_train, y_train)}')
    print(f'Test Score: {model.score(x_validation, y_validation)}') 
    
    y_pred = model.predict(x_validation)
    print(f'confusion_matrix:\n {confusion_matrix(y_validation, y_pred)}')
    print(f'accuracy_score: {accuracy_score(y_validation, y_pred)}')
    print(f'precision_score: {precision_score(y_validation, y_pred,average="macro")}')
    print(f'recall_score: {recall_score(y_validation, y_pred,average="macro")}')
    print(f'f1_score: {recall_score(y_validation, y_pred,average="macro")}')
    print('-------------')

Using model: AdaBoost
Training Score: 1.0
Test Score: 0.6346880907372401
confusion_matrix:
 [[1154  404]
 [ 369  189]]
accuracy_score: 0.6346880907372401
precision_score: 0.5382167086129599
recall_score: 0.5397014369125015
f1_score: 0.5397014369125015
-------------
Using model: GradientBoosting
Training Score: 0.7607847772130953
Test Score: 0.7457466918714556
confusion_matrix:
 [[1536   22]
 [ 516   42]]
accuracy_score: 0.7457466918714556
precision_score: 0.7023940058479532
recall_score: 0.5305740748409182
f1_score: 0.5305740748409182
-------------
Using model: BaggingClassifier
Training Score: 0.974707481385179
Test Score: 0.717391304347826
confusion_matrix:
 [[1442  116]
 [ 482   76]]
accuracy_score: 0.717391304347826
precision_score: 0.5726567914067914
recall_score: 0.5308731440455321
f1_score: 0.5308731440455321
-------------
Using model: RandomForest
Training Score: 0.9387779222314148
Test Score: 0.6658790170132325
confusion_matrix:
 [[1273  285]
 [ 422  136]]
accuracy_score: 0.66

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    max_depth = trial.suggest_int("max_depth", 1, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    
    # Create and train the model
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42
    )
    
    # Use cross-validation to evaluate the model
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy")
    return scores.mean()

# Create a study and optimize
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

In [86]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit and evaluate models
models, predictions = clf.fit(x_train, x_validation, y_train, y_validation)




100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [01:23<00:00,  2.87s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
NearestCentroid                    0.59               0.57     0.57      0.62   
XGBClassifier                      0.72               0.55     0.55      0.68   
DecisionTreeClassifier             0.65               0.55     0.55      0.65   
LGBMClassifier                     0.73               0.54     0.54      0.68   
Perceptron                         0.62               0.54     0.54      0.63   
BaggingClassifier                  0.73               0.54     0.54      0.67   
BernoulliNB                        0.73               0.54     0.54      0.67   
AdaBoostClassifier                 0.74               0.53     0.53      0.67   
PassiveAggressiveClassifier        0.64               0.53     0.53      0.64   
KNeighborsClassifier               0.70               0.53     0.53      0.66   
LabelPropagation            

In [87]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.59,0.57,0.57,0.62,0.13
XGBClassifier,0.72,0.55,0.55,0.68,0.82
DecisionTreeClassifier,0.65,0.55,0.55,0.65,0.20
LGBMClassifier,0.73,0.54,0.54,0.68,0.71
Perceptron,0.62,0.54,0.54,0.63,0.17
BaggingClassifier,0.73,0.54,0.54,0.67,0.96
BernoulliNB,0.73,0.54,0.54,0.67,0.09
AdaBoostClassifier,0.74,0.53,0.53,0.67,0.84
PassiveAggressiveClassifier,0.64,0.53,0.53,0.64,0.21
